In [1]:
from numba import cuda
import torch
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)


cuda.select_device(0)
cuda.close()
cuda.select_device(0)

torch.cuda.empty_cache()


df = pd.read_csv("15_new_DS.csv")
df.head()


,path,sentence
0,/home/peterr/macocu/task6_speech/data/00000001.flac.wav,poštovane kolegice i kolege zastupnici molio bi da nastavimo sa radom sjednice
1,/home/peterr/macocu/task6_speech/data/00000002.flac.wav,prvi klub koji se javio za stanku je klub esdepea
2,/home/peterr/macocu/task6_speech/data/00000003.flac.wav,u njihovo ime predstavnik bojan glavašević
3,/home/peterr/macocu/task6_speech/data/00000004.flac.wav,hvala gospodine predsjedniče
4,/home/peterr/macocu/task6_speech/data/00000005a.flac.wav,poštovani gospodine predsjedniče hrvatskog sabora uvažene kolegice i kolege klub zastupnika esdepea


In [2]:
LIM = int(df.shape[0] * 0.8)
common_voice_test_df = df.loc[LIM:, :].copy().reset_index(drop=True)

In [3]:

import datasets
from datasets import load_dataset, load_metric, Audio
def load_audio(path):
    return datasets.Audio(sampling_rate=16000).decode_example(path)

#common_voice_train_df.loc[:, "audio"] = common_voice_train_df.path.apply(load_audio)
common_voice_test_df.loc[:, "audio"] = common_voice_test_df.path.apply(load_audio)

#common_voice_train_dataset = datasets.Dataset.from_pandas(common_voice_train_df)
common_voice_test_dataset = datasets.Dataset.from_pandas(common_voice_test_df)


from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(
    "./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

feature_extractor = Wav2Vec2FeatureExtractor(
    feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

processor = Wav2Vec2Processor(
    feature_extractor=feature_extractor, tokenizer=tokenizer)





from transformers import Trainer
from transformers import TrainingArguments
from transformers import Wav2Vec2ForCTC
from typing import Any, Dict, List, Optional, Union
from dataclasses import dataclass, field
import torch


def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(
        audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch


#common_voice_train_mapped = common_voice_train_dataset.map(
    #prepare_dataset, remove_columns=common_voice_train_dataset.column_names)
common_voice_test_mapped = common_voice_test_dataset.map(
    prepare_dataset, remove_columns=common_voice_test_dataset.column_names)

file ./config.json not found
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


  0%|          | 0/7464 [00:00<?, ?ex/s]

In [4]:
repo_name = "15_/checkpoint-13600"
import logging
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(message)s', level=logging.INFO)
model = Wav2Vec2ForCTC.from_pretrained(repo_name)#.to("cuda")

rezs = list()
for i in common_voice_test_df.index:
    try:
        input_dict = processor(common_voice_test_mapped[i]["input_values"], return_tensors="pt", padding=True,
        sampling_rate=16000)
        #logits = model(input_dict.input_values.to("cuda")).logits
        logits = model(input_dict.input_values).logits
        pred_ids = torch.argmax(logits, dim=-1)[0]

        gold = common_voice_test_df.sentence[i]   
        prediction = processor.decode(pred_ids)
        fullpath = common_voice_test_df.path[i]
        filename = fullpath.split("/")[-1]
        rezs.append(prediction)
    except Exception as e:
        logging.error(f"Got exception: {e} for {common_voice_test_df.loc[i, 'path']}")
        break
df[repo_name] = rezs

In [ ]:
repo_name = "classla/wav2vec2-xls-r-sabor-hr"
import logging
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(message)s', level=logging.INFO)
model = Wav2Vec2ForCTC.from_pretrained(repo_name)#.to("cuda")

rezs = list()
for i in common_voice_test_df.index:
    try:
        input_dict = processor(common_voice_test_mapped[i]["input_values"], return_tensors="pt", padding=True,
        sampling_rate=16000)
        #logits = model(input_dict.input_values.to("cuda")).logits
        logits = model(input_dict.input_values).logits
        pred_ids = torch.argmax(logits, dim=-1)[0]

        gold = common_voice_test_df.sentence[i]   
        prediction = processor.decode(pred_ids)
        fullpath = common_voice_test_df.path[i]
        filename = fullpath.split("/")[-1]
        rezs.append(prediction)
    except Exception as e:
        logging.error(f"Got exception: {e} for {common_voice_test_df.loc[i, 'path']}")
        break
df[repo_name] = rezs

In [ ]:
df.to_csv("15_comparison_of_models.csv", index=False)